<a href="https://colab.research.google.com/github/june1st/GoogleColab/blob/master/SuperSlomo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Super Slomo
https://github.com/avinashpaliwal/Super-SloMo

In [1]:
!git clone https://github.com/avinashpaliwal/Super-SloMo

Cloning into 'Super-SloMo'...
remote: Enumerating objects: 135, done.
remote: Total 135 (delta 0), reused 0 (delta 0), pack-reused 135
Receiving objects: 100% (135/135), 26.87 MiB | 8.20 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [2]:
%cd Super-SloMo/

/content/Super-SloMo


In [3]:
ls

data/          eval.py  misc/     README.md    train.py
dataloader.py  LICENSE  model.py  train.ipynb  video_to_slomo.py*


In [18]:
# ffmpeg 
!apt install ffmpeg > /dev/null
!which ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.4-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [23]:
!ls data

adobe240fps  create_dataset.py


In [24]:
# Download dataset
!wget http://www.cs.ubc.ca/labs/imager/tr/2017/DeepVideoDeblurring/DeepVideoDeblurring_Dataset_Original_High_FPS_Videos.zip
!unzip DeepVideoDeblurring_Dataset_Original_High_FPS_Videos.zip

--2019-04-15 10:19:31--  http://www.cs.ubc.ca/labs/imager/tr/2017/DeepVideoDeblurring/DeepVideoDeblurring_Dataset_Original_High_FPS_Videos.zip
Resolving www.cs.ubc.ca (www.cs.ubc.ca)... 142.103.6.5
Connecting to www.cs.ubc.ca (www.cs.ubc.ca)|142.103.6.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2995547015 (2.8G) [application/zip]
Saving to: ‘DeepVideoDeblurring_Dataset_Original_High_FPS_Videos.zip’

DeepVideoDeblurring 100%[===================>]   2.79G  15.7MB/s    in 3m 27s  

2019-04-15 10:22:59 (13.8 MB/s) - ‘DeepVideoDeblurring_Dataset_Original_High_FPS_Videos.zip’ saved [2995547015/2995547015]

Archive:  DeepVideoDeblurring_Dataset_Original_High_FPS_Videos.zip
   creating: original_high_fps_videos/
  inflating: original_high_fps_videos/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/original_high_fps_videos/
  inflating: __MACOSX/original_high_fps_videos/._.DS_Store  
  inflating: original_high_fps_videos/720p_240fps_1.mov  
  inflating: or

In [0]:
!mkdir dataset_temp

In [0]:
!python data/create_dataset.py --ffmpeg_dir /usr/bin/ --videos_folder original_high_fps_videos --dataset_folder dataset_temp --dataset data/adobe240fps

ffmpeg version 3.4.4-0ubuntu0.18.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
#!rm -rf original_high_fps_videos/extracted
#!rm -rf original_high_fps_videos/train
#!rm -rf original_high_fps_videos/test
#!rm -rf original_high_fps_videos/validation

In [0]:
%cd data
!python create_dataset.py --ffmpeg_dir /usr/bin/ffmpeg --videos_folder ../original_high_fps_videos --dataset_folder ../original_high_fps_videos --dataset adobe240fps

In [26]:
ls

data/
dataloader.py
DeepVideoDeblurring_Dataset_Original_High_FPS_Videos.zip
disclaimer.txt
eval.py
LICENSE
log/
__MACOSX/
misc/
model.py
original_high_fps_videos/
__pycache__/
README.md
train.ipynb
train.py
video_to_slomo.py*


In [7]:
!pip install tensorboardX

    100% |████████████████████████████████| 133kB 3.9MB/s 


In [0]:
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import model
import dataloader
import matplotlib.pyplot as plt
from math import log10
from IPython.display import clear_output, display
import datetime
from tensorboardX import SummaryWriter

In [0]:
!mkdir checkPoint

In [0]:
# Learning Rate. Set `MILESTONES` to epoch values where you want to decrease
# learning rate by a factor of 0.1
INITIAL_LEARNING_RATE = 0.0001
MILESTONES = [100, 150]

# Number of epochs to train
EPOCHS = 200

# Choose batchsize as per GPU/CPU configuration
# This configuration works on GTX 1080 Ti
TRAIN_BATCH_SIZE = 6
VALIDATION_BATCH_SIZE = 10

# Path to dataset folder containing train-test-validation folders
DATASET_ROOT = "dataset_temp"#"path/to/dataset"

# Path to folder for saving checkpoints
CHECKPOINT_DIR = 'checkPoint'#'path/to/checkpoint_directory'

# If resuming from checkpoint, set `trainingContinue` to True and set `checkpoint_path`
TRAINING_CONTINUE = False
CHECKPOINT_PATH = 'checkPoint/cp'#'path/to/checkpoint/file'

# Progress and validation frequency (N: after every N iterations)
PROGRESS_ITER = 100

# Checkpoint frequency (N: after every N epochs). Each checkpoint is roughly of size 151 MB.
CHECKPOINT_EPOCH = 5

In [0]:
writer = SummaryWriter('log')

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
flowComp = model.UNet(6, 4)
flowComp.to(device)
ArbTimeFlowIntrp = model.UNet(20, 5)
ArbTimeFlowIntrp.to(device)

UNet(
  (conv1): Conv2d(20, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv2): Conv2d(32, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (down1): down(
    (conv1): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (down2): down(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (down3): down(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (down4): down(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (down5): down(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Co

In [0]:
trainFlowBackWarp      = model.backWarp(352, 352, device)
trainFlowBackWarp      = trainFlowBackWarp.to(device)
validationFlowBackWarp = model.backWarp(640, 352, device)
validationFlowBackWarp = validationFlowBackWarp.to(device)

In [13]:
# Channel wise mean calculated on adobe240-fps training dataset
mean = [0.429, 0.431, 0.397]
std  = [1, 1, 1]
normalize = transforms.Normalize(mean=mean,
                                 std=std)
transform = transforms.Compose([transforms.ToTensor(), normalize])

trainset = dataloader.SuperSloMo(root=DATASET_ROOT + '/train', transform=transform, train=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)

validationset = dataloader.SuperSloMo(root=DATASET_ROOT + '/validation', transform=transform, randomCropSize=(640, 352), train=False)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=VALIDATION_BATCH_SIZE, shuffle=False)

print(trainset, validationset)

FileNotFoundError: ignored